In [2]:
import json
import pandas as pd
import numpy as np
import mysql.connector
import os

In [24]:
# Getting the users data from database_init.json (add it to .gitignore)
with open('database_init.json') as file:
    db_config = json.load(file)
#creating NBA_DB if it is not already exists.
db_name = "NBA_DB"
try:
    temp_config = db_config.copy()
    cnxn = mysql.connector.connect(**temp_config)
    cursor = cnxn.cursor()

    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name} COLLATE utf8mb4_unicode_ci")
    print(f"Database '{db_name}' created or already exists.")
    cnxn.database = db_name


except mysql.connector.Error as err:
    print(f"Error!!: {err}")
finally:
    if 'cnxn' in locals() and cnxn.is_connected():
        cursor.close()
        cnxn.close()
        print("Connection Closed.")

Database 'NBA_DB' created or already exists.
Connection Closed.


In [90]:
players = pd.DataFrame()
#adding season year to columns and extracting player_id
lst = []
for i in {2019, 2020, 2021, 2022, 2023, 2024, 2025}:
    path = os.path.join('top 50 player of mjt list', f'nba_top50_players_{i}.csv')
    tmp = pd.read_csv(path)
    tmp['Season'] = i
    lst.append(tmp)
players = pd.concat(lst, axis=0, ignore_index=True)
players['player_id'] = players['Player_Link'].str.extract(r'/./(.*)\.html')
players.drop(columns='Player_Link', inplace=True)

In [106]:
players['MVP ranking'] = players['Awards'].str.extract(r'MVP-([0-9]*)').astype('Int64')

In [107]:
mvp_players = players[['player_id', 'MVP ranking', 'Season', 'Pos']].copy()
mvp_players.dropna(inplace=True)
mvp_players


,player_id,MVP ranking,Season,Pos
0,hardeja01,2,2019,PG
1,georgpa01,3,2019,SF
4,lillada01,6,2019,PG
5,duranke01,8,2019,SF
6,antetgi01,1,2019,PF
...,...,...,...,...
343,curryst01,9,2025,PG
345,jamesle01,6,2025,SF
346,mitchdo01,5,2025,SG
347,brunsja01,10,2025,PG


In [108]:
tbl_name = "AWARDS"
try:
    cnxn = mysql.connector.connect(**db_config)
    cursor = cnxn.cursor()
    cnxn.database = db_name

    cursor.execute(f"CREATE TABLE IF NOT EXISTS {tbl_name} ("
                   f"player_id VARCHAR(255) NOT NULL,"
                   f"Ranking INT NOT NULL,"
                   f"POS VARCHAR(255) NOT NULL,"
                   f"SEASON INT NOT NULL);")
    print(f"Table '{tbl_name}' created or already exists.")
    # Insert Dataframe into SQL Server:
    for index, row in mvp_players.iterrows():
        # print(f"inserting player {row['player_id']} year {row['Season']}")
        query = f"INSERT INTO {tbl_name} (player_id, Ranking, POS, SEASON) VALUES (%s, %s, %s, %s)"
        cursor.execute(query, (row['player_id'], row['MVP ranking'], row['Pos'], row['Season']))
    cnxn.commit()

except mysql.connector.Error as err:
    print(f"Error!!: {err}")
finally:
    if 'cnxn' in locals() and cnxn.is_connected():
        cursor.close()
        cnxn.close()
        print("Connection Closed.")

Table 'AWARDS' created or already exists.
Connection Closed.


In [120]:
top_players = players[['player_id', 'Rk', 'Age', 'Team', 'Pos', 'PTS', 'Season']].copy()
top_players

,player_id,Rk,Age,Team,Pos,PTS,Season
0,hardeja01,1,29,HOU,PG,2818,2019
1,georgpa01,2,28,OKC,SF,2159,2019
2,walkeke02,3,28,CHO,PG,2102,2019
3,bealbr01,4,25,WAS,SG,2099,2019
4,lillada01,5,28,POR,PG,2067,2019
...,...,...,...,...,...,...,...
379,banede01,46,26,MEM,SG,1327,2025
380,mobleev01,47,23,CLE,PF,1316,2025
381,powelno01,48,31,LAC,SG,1306,2025
382,bridgmi02,49,26,CHO,PF,1300,2025


In [123]:
tbl_name = "TOP_PLAYERS"
try:
    cnxn = mysql.connector.connect(**db_config)
    cursor = cnxn.cursor()
    cnxn.database = db_name

    cursor.execute(f"CREATE TABLE IF NOT EXISTS {tbl_name} ("
                   f"player_id VARCHAR(255) NOT NULL,"
                   f"Rk INT NOT NULL,"
                   f"AGE INT NOT NULL,"
                   f"team_id VARCHAR(20) NOT NULL,"
                   f"POS VARCHAR(20) NOT NULL,"
                   f"PTS INT NOT NULL,"
                   f"SEASON INT NOT NULL);")
    print(f"Table '{tbl_name}' created or already exists.")
    # Insert Dataframe into SQL Server:
    for index, row in top_players.iterrows():
        query = f"INSERT INTO {tbl_name} (player_id, Rk, AGE, team_id, POS, PTS, SEASON) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(query, (
            row['player_id'],
            row['Rk'],
            row['Age'],
            row['Team'],
            row['Pos'],
            row['PTS'],
            row['Season']))
    cnxn.commit()

except mysql.connector.Error as err:
    print(f"Error!!: {err}")
finally:
    if 'cnxn' in locals() and cnxn.is_connected():
        cursor.close()
        cnxn.close()
        print("Connection Closed.")

Table 'TOP_PLAYERS' created or already exists.
Connection Closed.


In [124]:
players_list = pd.read_csv(os.path.join('player stat', 'nba_players_stat.csv'))
players_list

,link,Name,Position,Shoots,Team,Born,Experience,Height_Weight
0,https://www.basketball-reference.com/players/l...,Kyle Lowry,Point Guard,Right,Philadelphia 76ers,"March 25,\n 1986\n (Age: 39-185d)\n...",19 years,"6-0,196lb (183cm, 88kg)"
1,https://www.basketball-reference.com/players/p...,Michael Porter Jr.,Small Forward,Right,Brooklyn Nets,"June 29,\n 1998\n (Age: 27-089d)\n\...",6 years,"6-10,218lb (208cm, 98kg)"
2,https://www.basketball-reference.com/players/v...,Nikola Vučević,Center,Right,Chicago Bulls,"October 24,\n 1990\n (Age: 34-337d)...",14 years,"6-10,260lb (208cm, 117kg)"
3,https://www.basketball-reference.com/players/h...,Montrezl Harrell,Center and Power Forward,Right,NaN,"January 26,\n 1994\n (Age: 31-243d)...",8 years,"6-7,240lb (201cm, 108kg)"
4,https://www.basketball-reference.com/players/b...,Desmond Bane,Shooting Guard,Right,Orlando Magic,"June 25,\n 1998\n (Age: 27-093d)\n\...",5 years,"6-5,215lb (196cm, 97kg)"
...,...,...,...,...,...,...,...,...
110,https://www.basketball-reference.com/players/m...,Bennedict Mathurin,Small Forward and Shooting Guard,Right,Indiana Pacers,"June 19,\n 2002\n (Age: 23-099d)\n\...",3 years,"6-6,210lb (198cm, 95kg)"
111,https://www.basketball-reference.com/players/p...,Norman Powell,Shooting Guard and Small Forward,Right,Miami Heat,"May 25,\n 1993\n (Age: 32-124d)\n\n...",10 years,"6-3,215lb (190cm, 97kg)"
112,https://www.basketball-reference.com/players/b...,Devin Booker,Shooting Guard and Point Guard,Right,Phoenix Suns,"October 30,\n 1996\n (Age: 28-331d)...",10 years,"6-6,206lb (198cm, 93kg)"
113,https://www.basketball-reference.com/players/w...,Victor Wembanyama,Center and Power Forward,Right,San Antonio Spurs,"January 4,\n 2004\n (Age: 21-265d)\...",2 years,"7-3,235lb (221cm, 106kg)"


In [125]:
# players_list['Age'] = players_list['Born'].str.extract(r'\(Age.*([0-9].)-').astype('Int64')
players_list['id'] = players_list['link'].str.extract(r'/./(.*)\.html')
# players_list['Experience'] = players_list['Experience'].str.extract(r'([0-9].)').astype('Int64')
players_list['Height'] = players_list['Height_Weight'].str.extract(r'([0-9]*)cm').astype('Int64')
players_list['Weight'] = players_list['Height_Weight'].str.extract(r'([0-9]*)kg').astype('Int64')
players_list.drop(columns=['Born', 'link', 'Height_Weight'], inplace=True)

In [137]:
players_list.drop_duplicates(inplace=True)

In [128]:
tbl_name = "PLAYERS_DETAIL"
try:
    cnxn = mysql.connector.connect(**db_config)
    cursor = cnxn.cursor()
    cnxn.database = db_name

    cursor.execute(f"CREATE TABLE IF NOT EXISTS {tbl_name} ("
                   f"player_id VARCHAR(255) NOT NULL PRIMARY KEY,"
                   f"NAME VARCHAR(255) NOT NULL,"
                   f"POS VARCHAR(255) NOT NULL,"
                   f"SHOOTS VARCHAR(10) NOT NULL,"
                   # f"TEAM VARCHAR(255),"
                   f"EXPERIENCE INT,"
                   # f"AGE INT NOT NULL,"
                   f"HEIGHT INT NOT NULL,"
                   f"WEIGHT INT NOT NULL);")
    print(f"Table '{tbl_name}' created or already exists.")
    # Insert Dataframe into SQL Server:
    for index, row in players_list.iterrows():
        query = f"INSERT INTO {tbl_name} (player_id, NAME, POS, EXPERIENCE, SHOOTS,  HEIGHT, WEIGHT) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(query, (
            row['id'],
            row['Name'],
            row['Position'],
            row['Shoots'],
            # row['Team'] if pd.notna(row['Team']) else None,
            row['Experience'] if pd.notna(row['Experience']) else None,
            # row['Age'],
            row['Height'],
            row['Weight']))
    cnxn.commit()

except mysql.connector.Error as err:
    print(f"Error!!: {err}")
finally:
    if 'cnxn' in locals() and cnxn.is_connected():
        cursor.close()
        cnxn.close()
        print("Connection Closed.")

Table 'PLAYERS_DETAIL' created or already exists.
Connection Closed.


In [16]:
file_names = os.listdir('champ team')
lst = []
for file in file_names:
    if file[-2:] != "py":  # escaping python files
        path = os.path.join('champ team', file)
        tmp = pd.read_csv(path)
        lst.append(tmp)

winners = pd.concat(lst, axis=0, ignore_index=True)
winners

,No.,Player,Pos,Ht,Wt,Birth Date,Birth,Exp,College,player_link,team_id,year
0,0,Christian Braun,SG,6-6,218,"April 17, 2001",usUS,R,Kansas,https://www.basketball-reference.com/players/b...,DEN,2023
1,11,Bruce Brown,SF,6-4,202,"August 15, 1996",usUS,4,Miami (FL),https://www.basketball-reference.com/players/b...,DEN,2023
2,13,Thomas Bryant,C,6-10,248,"July 31, 1997",usUS,5,Indiana,https://www.basketball-reference.com/players/b...,DEN,2023
3,5,Kentavious Caldwell-Pope,SG,6-5,204,"February 18, 1993",usUS,9,Georgia,https://www.basketball-reference.com/players/c...,DEN,2023
4,31,Vlatko Čančar,PF,6-8,236,"April 10, 1997",siSI,3,NaN,https://www.basketball-reference.com/players/c...,DEN,2023
...,...,...,...,...,...,...,...,...,...,...,...,...
132,22,Malachi Richardson,SG,6-6,205,"January 5, 1996",usUS,2,Syracuse,https://www.basketball-reference.com/players/r...,TOR,2019
133,43,Pascal Siakam,PF,6-8,230,"April 2, 1994",cmCM,2,New Mexico State,https://www.basketball-reference.com/players/s...,TOR,2019
134,17,Jonas Valančiūnas,C,6-11,265,"May 6, 1992",ltLT,6,NaN,https://www.basketball-reference.com/players/v...,TOR,2019
135,23,Fred VanVleet,PG,6-0,197,"February 25, 1994",usUS,2,Wichita State,https://www.basketball-reference.com/players/v...,TOR,2019


In [22]:
winners['player_id'] = winners['player_link'].str.extract(r'/./(.*)\.html')
winners['Exp'] = winners['Exp'].str.replace('R', '0').astype('Int64')

In [27]:
tbl_name = "WINNER_TEAMS"
try:
    cnxn = mysql.connector.connect(**db_config)
    cursor = cnxn.cursor()
    cnxn.database = db_name

    cursor.execute(f"CREATE TABLE IF NOT EXISTS {tbl_name} ("
                   f"team_id VARCHAR(20) NOT NULL,"
                   f"player_id VARCHAR(255) NOT NULL,"
                   f"POS VARCHAR(20) NOT NULL,"
                   f"EXPERIENCE INT,"
                   f"YEAR INT NOT NULL);")
    print(f"Table '{tbl_name}' created or already exists.")
    # Insert Dataframe into SQL Server:
    for index, row in winners.iterrows():
        query = f"INSERT INTO {tbl_name} (team_id, player_id, POS, EXPERIENCE, YEAR) VALUES (%s, %s, %s, %s, %s)"
        cursor.execute(query, (
            row['team_id'],
            row['player_id'],
            row['Pos'],
            row['Exp'],
            row['year']))
    cnxn.commit()

except mysql.connector.Error as err:
    print(f"Error!!: {err}")
finally:
    if 'cnxn' in locals() and cnxn.is_connected():
        cursor.close()
        cnxn.close()
        print("Connection Closed.")

Table 'WINNER_TEAMS' created or already exists.
Connection Closed.


In [28]:
teams_list = pd.read_csv(os.path.join('team stat', 'nba_teams.csv'))
teams_list

,id,Team,Location,Seasons,Record,Playoff Appearances,Championships
0,OKC,Oklahoma City Thunder,"Oklahoma City, Oklahoma",59;\n \n 1967-68 to 2025-26,"2538-2150, .541 W-L%",34,2
1,IND,Indiana Pacers,"Indianapolis, Indiana",59 (50 NBA & 9 ABA);\n \n 1967-68 to 2025-26,"2407-2287, .513 W-L%\n \n (1980-1970 NBA &...",38\n \n (29 NBA & 9 ABA),3\n \n (0 NBA & 3 ABA)
2,PHI,Philadelphia 76ers,"Philadelphia, Pennsylvania",77;\n \n 1949-50 to 2025-26,"3125-2898, .519 W-L%",54,3
3,HOU,Houston Rockets,"Houston, Texas",59;\n \n 1967-68 to 2025-26,"2421-2267, .516 W-L%",35,2
4,POR,Portland Trail Blazers,"Portland, Oregon",56;\n \n 1970-71 to 2025-26,"2328-2116, .524 W-L%",37,1
5,SAS,San Antonio Spurs,"San Antonio, Texas",59 (50 NBA & 9 ABA);\n \n 1967-68 to 2025-26,"2717-1976, .579 W-L%\n \n (2339-1610 NBA &...",47\n \n (39 NBA & 8 ABA),5\n \n (5 NBA & 0 ABA)
6,BOS,Boston Celtics,"Boston, Massachusetts",80;\n \n 1946-47 to 2025-26,"3695-2501, .596 W-L%",62,18
7,NOP,New Orleans Pelicans,"New Orleans, Louisiana",24;\n \n 2002-03 to 2025-26,"852-998, .461 W-L%",9,0
8,MIN,Minnesota Timberwolves,"Minneapolis, Minnesota",37;\n \n 1989-90 to 2025-26,"1196-1680, .416 W-L%",13,0
9,ATL,Atlanta Hawks,"Atlanta, Georgia",77;\n \n 1949-50 to 2025-26,"2967-3052, .493 W-L%",49,1


In [30]:
tbl_name = "TEAMS_DETAILS"
try:
    cnxn = mysql.connector.connect(**db_config)
    cursor = cnxn.cursor()
    cnxn.database = db_name

    cursor.execute(f"CREATE TABLE IF NOT EXISTS {tbl_name} ("
                   f"id VARCHAR(20) NOT NULL PRIMARY KEY,"
                   f"NAME VARCHAR(255) NOT NULL,"
                   f"LOCATION VARCHAR(255) NOT NULL);")
    print(f"Table '{tbl_name}' created or already exists.")
    # Insert Dataframe into SQL Server:
    for index, row in teams_list.iterrows():
        query = f"INSERT INTO {tbl_name} (id, NAME, LOCATION) VALUES (%s, %s, %s)"
        cursor.execute(query, (
            row['id'],
            row['Team'],
            row['Location']))
    cnxn.commit()

except mysql.connector.Error as err:
    print(f"Error!!: {err}")
finally:
    if 'cnxn' in locals() and cnxn.is_connected():
        cursor.close()
        cnxn.close()
        print("Connection Closed.")

Table 'TEAMS_DETAILS' created or already exists.
Connection Closed.
